In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
%matplotlib inline
from pathlib import Path
import functools
import itertools
from dataclasses import dataclass, fields

import pandas as pd
import numpy as np

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
from matplotlib import rcParams 

In [4]:
from lib import datawrap #E
from lib import bar # artist
from lib import color, position # encoders


#spath = Path("../../../draft/figures/code/")
rcParams['figure.dpi'] = 200

In [5]:
data = {'fruit': ['apple', 'orange', 'lemon', 'lime'], 
       'calories':[95, 67, 17, 20], 
       'juice':[True, True, False, False]}
df = pd.DataFrame(data)

In [6]:
data_bounds = {('calories',):((20, 100), ), 
               ('juice', ): (None, ), 
               ('fruit',):(['orange', 'lemon', 'lime'], )}
tau = datawrap.FruitFrameWrapper(df)

In [7]:
tau.query(data_bounds)

[{'fruit': array(['orange', 'lime'], dtype=object),
  'calories': array([67, 20], dtype=int64),
  'juice': array([ True, False])}]

In [85]:
# should K be up front or last?
def identity(val):
     return val

def constant_color(c):
     return lambda x: [mcolors.to_rgba(c) for _ in range(x)]

def constant_value(val):
     return lambda x: [val for _ in range(x)]

def composite_x_transform(ax, nu):
     def f(x):
          return [ax.transData.transform((xi, 0))[0] for xi in nu(x)]
     return f

def composite_y_transform(ax, nu):
     def f(y):
          return [ax.transData.transform((0, yi))[1] for yi in nu(y)]
     return f

def composite_x_distance_transform(ax, nu):
     def f(x):
          return [np.abs(ax.transData.transform((xi, 0)) 
                    - ax.transData.transform((0,0)))[0] for xi in nu(x)]
     return f

def composite_y_distance_transform(ax, nu):
     def f(y):
          return [np.abs(ax.transData.transform((xi, 0)) 
                    - ax.transData.transform((0,0)))[1] for yi in nu(y)]
     return f

In [86]:
# multivariate nu, make key always a tuple ([possibly data class])
fruitPos = position.Nominal({'apple': 0, 'orange': 2, 'lemon': 4, 'lime': 6})
juiceCol = color.Nominal({True: 'navy', False: 'purple'})
fruitCol = color.Nominal({'apple': 'xkcd:red', 'orange': 'orange', 
                         'lemon': 'gold', 'lime': 'lime'})

In [89]:
def make_V(ax, h=False):
    V = [('length', ('calories',), composite_y_transform(ax, identity), None), 
            ('position', ('fruit',), composite_x_transform(ax, fruitPos), fruitPos.inverse),
            ('floor', None,  composite_y_transform(ax, constant_value(0)), None), 
            ('width', None,  composite_x_distance_transform(ax, constant_value(.8)), None),
            ('edgecolor', ('juice',), juiceCol, juiceCol.inverse),
            ('facecolor', ('fruit',), fruitCol, fruitCol.inverse),
            ('linewidth', None, constant_value(5), None),
             ('linestyle', None, constant_value((0, None)), None)]
    if h:
        V[0] = ('position', ('calories', ), composite_x_transform(ax, identity), None)
        V[1] = ('length', ('fruit'), composite_y_transform(ax, fruitPos), fruitPos.inverse)
    return V

In [93]:
fig, ax = plt.subplots()
dbar = bar.Bar()
for (p, f, nu, nu_inv) in make_V(ax):
  pbar = dbar.compose_with_nu(p, f, nu, nu_inv)
A = bar.TopologicalArtist(pbar).compose_with_tau(tau)

axesArtist = bar.GenericArtist(A)
ax.add_artist(axesArtist)
ax.set(xlim=(-1, 10 ), ylim=(0, 110))

[(-1.0, 10.0), (0.0, 110.0)]

KeyError: 'height'

KeyError: 'height'

<Figure size 1200x800 with 1 Axes>

In [62]:
axesArtist.artist.graphic.mu(axesArtist.artist.data.query()[0])

{'x': [234.54545454545456,
  403.6363636363636,
  572.7272727272727,
  741.8181818181818],
 'y': [621.6363636363636,
  467.8909090909091,
  193.34545454545452,
  209.8181818181818],
 'facecolor': [RGBA(r=0.8980392156862745, g=0.0, b=0.0, a=1.0),
  RGBA(r=1.0, g=0.6470588235294118, b=0.0, a=1.0),
  RGBA(r=1.0, g=0.8431372549019608, b=0.0, a=1.0),
  RGBA(r=0.0, g=1.0, b=0.0, a=1.0)],
 'edgecolor': [RGBA(r=0.0, g=0.0, b=0.5019607843137255, a=1.0),
  RGBA(r=0.0, g=0.0, b=0.5019607843137255, a=1.0),
  RGBA(r=0.5019607843137255, g=0.0, b=0.5019607843137255, a=1.0),
  RGBA(r=0.5019607843137255, g=0.0, b=0.5019607843137255, a=1.0)],
 'width': [234.5455390909091,
  234.5455390909091,
  234.5455390909091,
  234.5455390909091],
 'linewidth': [5, 5, 5, 5],
 'linestyle': [(0, None), (0, None), (0, None), (0, None)],
 'floor': [100.0, 100.0, 100.0, 100.0]}

In [78]:
ax.transData.transform((0,.8)) - ax.transData.transform((0,0)) 

array([0.        , 4.39272727])

In [74]:
import matplotlib.image as mimage
from matplotlib.backends.backend_agg import RendererAgg

# this is what GenericArtist is doing

renderer = RendererAgg(500, 400, 100)
bounds, dpi = axesArtist.get_screen_bounds_to_data_bounds(renderer)
tau_r = axesArtist.artist.data.query(bounds, dpi)
for tr in tau_r:
    print (tr)
    mu = axesArtist.artist.graphic.mu(tr)
    #rho = axesArtist.artist.graphic.qhat(**mu)
    #H = rho(renderer)

mimage.imsave('bar2.png', renderer.buffer_rgba(),
              format='png', origin='upper', dpi=100)


{'fruit': array(['apple', 'orange', 'lemon', 'lime'], dtype=object), 'calories': array([95, 67, 17, 20], dtype=int64), 'juice': array([ True,  True, False, False])}


In [ ]:
fig, ax = plt.subplots()
dbarh = bar.BarH()
for (pi, fi, nui, nu_inv) in make_V(ax, h=True):
  pbarh = dbarh.compose_with_nu(pi, fi, nui)
Ah = bar.TopologicalArtist(pbarh).compose_with_tau(tau)

ax.add_artist(bar.GenericArtist(Ah))
ax.set(ylim=(-1,8), xlim=(0, 100))

In [ ]:
from matplotlib.backends.backend_agg import RendererAgg
renderer = RendererAgg(500, 400, 100)
axesArtist.get_screen_bounds_to_data_bounds(renderer)

In [ ]:
fig, ax = plt.subplots()
axesArtist = bar.GenericArtist(A)
ax.add_artist(axesArtist)
ax.set(xlim=(-1,8), ylim=(0, 100))